In [1]:
import collections
import copy
import datetime
import itertools
import sys

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay

In [3]:
G.register_tournaments()

In [4]:
G.register_matchups()

In [5]:
G.register_achievements()

In [6]:
tournaments = sorted(G.tournaments.values())

In [7]:
i = 18

In [18]:
T = tournaments[i]

In [19]:
print(f'{i} [{T.Id}] {T.name}')

20 [42806] Y2, Winter - Friendlies (Low TW)


In [20]:
s = cibblbibbl.tournament.export.awards.plaintext.export(T, show_Ids=True)
print(s)

25th place: [872802] Grey Hag Hazzadurrs
24th place: [872948] Volkn‌‌ Firetails (Fake)
23th place: [871123] Knights of the Black Grail
22th place: [873245] Sphinx Legion of Numas
21th place: [872809] Crookback Pit Evil Moons
20th place: [872044] Strigos Elită
19th place: [872710] Drakenhof Dance Macabre
18th place: [871787] Malignant Curators
17th place: [873369] Schattenwald Drinking Horns
16th place: [873478] Barak Varr Ironbreakers
15th place: [873391] Deep Sea Bottom Feeders
14th place: [872927] Notorious BIBBL.Inc.Brawlers
13th place: [873691] Volkn‌‌ Firetails
12th place: [872524] Silver Pinnacle Renegades
11th place: [870773] Coeth-Mara Warhawks
10th place: [873081] Cavernous Crystalbreakers
9th place: [873361] Blood-Ground Blackhorns
8th place: [872949] Karak Kadrin Honey Badgers
7th place: [873015] Fumes of our Farthers
6th place: [873349] Reik River Kicking Mules
5th place: [873152] Go-Go Gadget Gnomes
4th place: [872804] Zutzendorf Stallions
3rd place: [869156] Talsyn Trampl

In [21]:
for A in T.achievements:
    if A["status"] == "proposed":
        A["status"] = "awarded"

In [22]:
i += 1

In [11]:
PO = "passed over, not awarded"

for A in T.achievements:
    if A["status"] == "proposed":
        print(A)
        print(A.subject.name)
        sys.stdout.flush()
        while True:
            answer = input("(a)warded/(p)assed over/(r)ejected (default: awarded)?").lower()
            if answer in ("", "a"):
                A["status"] = "awarded"
                break
            elif answer in ("p"):
                A["status"] = PO
                break
            elif answer in ("r"):
                A["status"] = "rejected"
                break

Child of Winter: Helmut Wulf (Star Player) (Fouls)
Helmut Wulf
(a)warded/(p)assed over/(r)ejected (default: awarded)?p
Child of Winter: Jessie Graff (Ninja Warriors) (Completions)
Jessie Graff
(a)warded/(p)assed over/(r)ejected (default: awarded)?
Child of Winter: Mosstail (Seven Samples of Decay) (SPP)
Mosstail
(a)warded/(p)assed over/(r)ejected (default: awarded)?
Child of Winter: Tutencal the Harvester (Guardians of the Lurking God) (Touchdowns)
Tutencal the Harvester
(a)warded/(p)assed over/(r)ejected (default: awarded)?
Winter's Glory: Ninja Warriors
Ninja Warriors
(a)warded/(p)assed over/(r)ejected (default: awarded)?
Child of Winter: Ronin Lethal Strike (Secret Cult Of Shiamoto) (Blocking Scorer, Casualties)
Ronin Lethal Strike
(a)warded/(p)assed over/(r)ejected (default: awarded)?
Winter's Glory: Lords of the Silent Death
Lords of the Silent Death
(a)warded/(p)assed over/(r)ejected (default: awarded)?
Child of Winter: Hazu (Secret Cult Of Shiamoto) (Blocking Thrower)
Hazu
(a)wa

In [ ]:
T.deadplayers()

In [27]:
not_dead = [
    [3893188, "STAR-Helmut Wulf"],
]

In [28]:
for matchId, playerId in not_dead:
    Ma = cibblbibbl.match.Match(matchId)
    Mu = Ma.matchup
    print(Mu)
    C = Mu.config
    for d0 in C["player"].values():
        for playerId1, d1 in d0.items():
            if playerId == playerId1:
                del d1["dead"]
                break
        else:
            continue
        break

Matchup('cibbl', '42748', 3, 872475, 872524)


In [25]:
admin_forfeits = [
    [["cibbl", "42041", 4, 855753, 856203], "856203"],
]

In [26]:
for matchup_KEY, teamId in admin_forfeits:
    Mu = Matchup(*matchup_KEY)
    d = Mu.config["team"][teamId]
    d["prestige"] = 0
    d["pts"] = 0
    d["r"] = "F"
    d["score"] = 0
    d["scorediff"] = -2
    d["tddiff"] = 0

In [28]:
excluded_matchups = [
    ["cibbl", "42239", 0, 863440, 863441],
    ["cibbl", "42239", 0, 863440, 864291],
    ["cibbl", "42239", 0, 863441, 864291],
]

In [29]:
for matchup_KEY in excluded_matchups:
    Mu = Matchup(*matchup_KEY)
    Mu.excluded = "yes"

In [22]:
print(cibblbibbl.tournament.export.standings.plaintext.export(T))

 #                Name                      Roster                 Coach           Perf.    PTS   HTH   TDD   CAD   CTO
 1   Tor Anroc Anglers                Sea Elf               neubau                DWBFW      W0           2     2      
 2   Doom of Athelorn                 Pirates of Sartosa    Mr_Foxx               DWFFW      70     1     7     2      
 3   Norscan Curse                    Undead Pirates        darkjul               LLBBD      70     2    -4     4      
 4   Blackmud Tridents                Nautican              frzntoz               LLD        10          -5    -8      


In [ ]:
print(cibblbibbl.tournament.export.prestiges.plaintext.export(T))

In [10]:
L = sorted(T.achievements)
L

[TP_Admin('tp_admin', Tournament('cibbl', '42428'), Team(854678)),
 TP_Admin('tp_admin', Tournament('cibbl', '42428'), Team(854600)),
 TP_Match('tp_match', Tournament('cibbl', '42428'), Team(855228)),
 TP_Match('tp_match', Tournament('cibbl', '42428'), Team(856697)),
 TP_Match('tp_match', Tournament('cibbl', '42428'), Team(854600)),
 TP_Match('tp_match', Tournament('cibbl', '42428'), Team(855120)),
 TP_Match('tp_match', Tournament('cibbl', '42428'), Team(854678)),
 TP_Match('tp_match', Tournament('cibbl', '42428'), Team(854257)),
 TP_Match('tp_match', Tournament('cibbl', '42428'), Team(859097)),
 TP_Standings('tp_standings', Tournament('cibbl', '42428'), Team(855228)),
 TP_Standings('tp_standings', Tournament('cibbl', '42428'), Team(856697)),
 TP_Standings('tp_standings', Tournament('cibbl', '42428'), Team(854600)),
 TP_Standings('tp_standings', Tournament('cibbl', '42428'), Team(855120)),
 PA_BPA_Gold('pa_bpa_gold', Tournament('cibbl', '42428'), NormalPlayer('11512476')),
 PA_BPA_Gold

In [13]:
L[1].prestige(T.season)

-20

In [31]:
Re = Replay(1040049)

In [32]:
with Re as Re:
    pprint(Re.normteamdata)

{Team(871312): {'apothecaries': 0,
                'assistantCoaches': 0,
                'baseIconPath': 'http://fumbbl.com/',
                'cheerleaders': 0,
                'coach': 'badenprinz',
                'fanFactor': 10,
                'logoUrl': 'i/517978',
                'playerArray': [{'agility': 4,
                                 'armour': 8,
                                 'lastingInjuries': ['Smashed Hand (MNG)'],
                                 'movement': 6,
                                 'nrOfIcons': 0,
                                 'playerGender': 'male',
                                 'playerId': '11986735',
                                 'playerName': 'Count Ludovig II.',
                                 'playerNr': 1,
                                 'playerType': 'Regular',
                                 'positionIconIndex': 0,
                                 'positionId': '7387',
                                 'recoveringInjury': None,
 

                                                             'Regeneration',
                                                             'Thick Skull',
                                                             'Throw Team-Mate'],
                                              'skillCategoriesDouble': ['Strength',
                                                                        'Agility',
                                                                        'General',
                                                                        'Passing'],
                                              'skillCategoriesNormal': [],
                                              'skillValues': [0, 0, 0, 0, 0],
                                              'strength': 5,
                                              'teamWithPositionId': None,
                                              'thrall': False,
                                              'undead': False,
                  

In [33]:
with Re as Re:
    print(Re.calculate_playerIdnorm())

{'888760M1': 'MERC-1040049-888760M1'}


In [37]:
Pl = self = player('MERC-1040049-888760M1')

In [36]:
Pl._name

'Merc Drone 1'

In [38]:
replayId, playerId = self.Id[5:].split("-")

In [39]:
Re = cibblbibbl.replay.Replay(int(replayId))

In [41]:
Te = cibblbibbl.team.Team(int(playerId.split("M")[0]))

In [49]:
with Re as Re:
    D = Re.teamdata[Te]

In [51]:
for d in D["playerArray"]:
    if d["playerId"] == playerId:
        break
else:
    raise Exception("not found")

In [52]:
d["playerName"]

'Merc Drone 1'

In [34]:
subject = Team(854600)

In [38]:
As = sorted(A for A in T.achievements if A.subject is subject)
As

[TP_Admin('tp_admin', Tournament('cibbl', '42428'), Team(854600)),
 TP_Match('tp_match', Tournament('cibbl', '42428'), Team(854600)),
 TP_Standings('tp_standings', Tournament('cibbl', '42428'), Team(854600))]

In [39]:
for A in As:
    print(A)

Admin Adjusted Prestige: Montetriste Noblemen (-20 Prestige Points)
Match Result Prestige: Montetriste Noblemen (90 Prestige Points)
Standings Prestige:  3. Montetriste Noblemen (50 Prestige Points)


In [40]:
As[0]

TP_Admin('tp_admin', Tournament('cibbl', '42428'), Team(854600))

In [17]:
for T in G.tournaments.values():
    if T.season.name == "Winter":
        T.config["bestplayersname"] = "Child of Winter"